In [1]:
import pandas as pd
import numpy as np

In [2]:
cohort_counts = pd.read_csv('../DATASETS/part4/cohort_counts.csv')
cohort_counts['CohortMonth'] = pd.to_datetime(cohort_counts['CohortMonth'])
cohort_counts.set_index(['CohortMonth'], inplace=True)
print(cohort_counts.head())

                 1      2      3      4      5      6      7      8      9  \
CohortMonth                                                                  
2010-12-01   716.0  246.0  221.0  251.0  245.0  285.0  249.0  236.0  240.0   
2011-01-01   332.0   69.0   82.0   81.0  110.0   90.0   82.0   86.0  104.0   
2011-02-01   316.0   58.0   57.0   83.0   85.0   74.0   80.0   83.0   86.0   
2011-03-01   388.0   63.0  100.0   76.0   83.0   67.0   98.0   85.0  107.0   
2011-04-01   255.0   49.0   52.0   49.0   47.0   52.0   56.0   59.0   17.0   

                10     11     12     13  
CohortMonth                              
2010-12-01   265.0  254.0  348.0  172.0  
2011-01-01   102.0  124.0   45.0    NaN  
2011-02-01    95.0   28.0    NaN    NaN  
2011-03-01    38.0    NaN    NaN    NaN  
2011-04-01     NaN    NaN    NaN    NaN  


In [3]:
# Extract cohort sizes from the first column of cohort_counts
cohort_sizes = cohort_counts.iloc[:,0]

# Calculate retention by dividing the counts with the cohort sizes
retention = cohort_counts.divide(cohort_sizes, axis=0)

# Calculate churn
churn = 1 - retention

# Print the retention table
print(retention)

               1         2         3         4         5         6         7  \
CohortMonth                                                                    
2010-12-01   1.0  0.343575  0.308659  0.350559  0.342179  0.398045  0.347765   
2011-01-01   1.0  0.207831  0.246988  0.243976  0.331325  0.271084  0.246988   
2011-02-01   1.0  0.183544  0.180380  0.262658  0.268987  0.234177  0.253165   
2011-03-01   1.0  0.162371  0.257732  0.195876  0.213918  0.172680  0.252577   
2011-04-01   1.0  0.192157  0.203922  0.192157  0.184314  0.203922  0.219608   
2011-05-01   1.0  0.160643  0.172691  0.144578  0.208835  0.232932  0.244980   
2011-06-01   1.0  0.159420  0.125604  0.198068  0.236715  0.299517  0.091787   
2011-07-01   1.0  0.161850  0.179191  0.219653  0.254335  0.098266       NaN   
2011-08-01   1.0  0.215827  0.201439  0.251799  0.100719       NaN       NaN   
2011-09-01   1.0  0.200717  0.279570  0.121864       NaN       NaN       NaN   
2011-10-01   1.0  0.210692  0.094340    

In [4]:
# Calculate the mean retention rate
retention_rate = retention.iloc[:,1:].mean().mean()

# Calculate the mean churn rate
churn_rate = churn.iloc[:,1:].mean().mean()

# Print rounded retention and churn rates
print('Retention rate: {:.2f}; Churn rate: {:.2f}'.format(retention_rate, churn_rate))

Retention rate: 0.24; Churn rate: 0.76


In [21]:
online = pd.read_csv('../DATASETS/part4/online.csv')
online['InvoiceDate']= pd.to_datetime(online['InvoiceDate'])
online['InvoiceMonth'] = online['InvoiceDate'].values.astype('datetime64[M]')
online['TotalSum'] = online['UnitPrice'] * online['Quantity']

online.head()

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceMonth,TotalSum
0,416792,572558,22745,POPPY'S PLAYHOUSE BEDROOM,6,2011-10-25 08:26:00,2.10,14286,United Kingdom,2011-10-01,12.60
1,482904,577485,23196,VINTAGE LEAF MAGNETIC NOTEPAD,1,2011-11-20 11:56:00,1.45,16360,United Kingdom,2011-11-01,1.45
2,263743,560034,23299,FOOD COVER WITH BEADS SET 2,6,2011-07-14 13:35:00,3.75,13933,United Kingdom,2011-07-01,22.50
3,495549,578307,72349B,SET/6 PURPLE BUTTERFLY T-LIGHTS,1,2011-11-23 15:53:00,2.10,17290,United Kingdom,2011-11-01,2.10
4,204384,554656,21756,BATH BUILDING BLOCK WORD,3,2011-05-25 13:36:00,5.95,17663,United Kingdom,2011-05-01,17.85


In [22]:
# Calculate monthly spend per customer
monthly_revenue = online.groupby(['CustomerID','InvoiceMonth'])['TotalSum'].sum()

# Calculate average monthly spend
monthly_revenue = np.mean(monthly_revenue)

# Define lifespan to 36 months
lifespan_months = 36

# Calculate basic CLV
clv_basic = monthly_revenue * lifespan_months

# Print the basic CLV value
print('Average basic CLV is {:.1f} USD'.format(clv_basic))

Average basic CLV is 4774.6 USD


In [23]:
# Calculate average revenue per invoice
revenue_per_purchase = online.groupby(['InvoiceNo'])['TotalSum'].mean().mean()

# Calculate average number of unique invoices per customer per month
frequency_per_month = online.groupby(['CustomerID','InvoiceMonth'])['InvoiceNo'].nunique().mean()

# Define lifespan to 36 months
lifespan_months = 36

# Calculate granular CLV
clv_granular = revenue_per_purchase * frequency_per_month * lifespan_months

# Print granular CLV value
print('Average granular CLV is {:.1f} USD'.format(clv_granular))

Average granular CLV is 1635.2 USD


In [24]:
# Calculate monthly spend per customer
monthly_revenue = online.groupby(['CustomerID','InvoiceMonth'])['TotalSum'].sum().mean()

# Calculate average monthly retention rate
retention_rate = retention.iloc[:,1:].mean().mean()

# Calculate average monthly churn rate
churn_rate = 1 - retention_rate

# Calculate traditional CLV 
clv_traditional = monthly_revenue * (retention_rate / churn_rate)

# Print traditional CLV and the retention rate values
print('Average traditional CLV is {:.1f} USD at {:.1f} % retention_rate'.format(clv_traditional, retention_rate*100))

Average traditional CLV is 42.4 USD at 24.2 % retention_rate


In [25]:
online_X = online[online['InvoiceMonth'] != '2011-11']
online_X.head()

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceMonth,TotalSum
0,416792,572558,22745,POPPY'S PLAYHOUSE BEDROOM,6,2011-10-25 08:26:00,2.10,14286,United Kingdom,2011-10-01,12.60
2,263743,560034,23299,FOOD COVER WITH BEADS SET 2,6,2011-07-14 13:35:00,3.75,13933,United Kingdom,2011-07-01,22.50
4,204384,554656,21756,BATH BUILDING BLOCK WORD,3,2011-05-25 13:36:00,5.95,17663,United Kingdom,2011-05-01,17.85
5,125615,547051,22028,PENNY FARTHING BIRTHDAY CARD,12,2011-03-20 12:06:00,0.42,12902,United Kingdom,2011-03-01,5.04
8,127438,547223,22934,BAKING MOULD EASTER EGG WHITE CHOC,2,2011-03-21 15:10:00,2.95,12867,United Kingdom,2011-03-01,5.90


In [26]:
import datetime as dt

In [27]:
# Define the snapshot date
NOW = dt.datetime(2011,11,1)

# Calculate recency by subtracting current date from the latest InvoiceDate
features = online_X.groupby('CustomerID').agg({
  'InvoiceDate': lambda x: (NOW - x.max()).days,
  # Calculate frequency by counting unique number of invoices
  'InvoiceNo': pd.Series.nunique,
  # Calculate monetary value by summing all spend values
  'TotalSum': np.sum,
  # Calculate average and total quantity
  'Quantity': ['mean', 'sum']}).reset_index()

# Rename the columns
features.columns = ['CustomerID', 'recency', 'frequency', 'monetary', 'quantity_avg', 'quantity_total']

In [28]:
# Build a pivot table counting invoices for each customer monthly
cust_month_tx = pd.pivot_table(data=online, values='InvoiceNo',
                               index=['CustomerID'], columns=['InvoiceMonth'],
                               aggfunc=pd.Series.nunique, fill_value=0)
print(cust_month_tx.head())

InvoiceMonth  2010-12-01  2011-01-01  2011-02-01  2011-03-01  2011-04-01  \
CustomerID                                                                 
12747                  2           1           0           1           0   
12748                 24           2           4           9           3   
12749                  0           0           0           0           0   
12820                  0           1           0           0           0   
12822                  0           0           0           0           0   

InvoiceMonth  2011-05-01  2011-06-01  2011-07-01  2011-08-01  2011-09-01  \
CustomerID                                                                 
12747                  2           1           0           1           0   
12748                 17          12           8           9           9   
12749                  1           0           0           1           0   
12820                  0           0           0           0           1   
12822      

In [36]:

NOW = dt.datetime(2011,11,1)
# Store November 2011 data column name as a list
target = NOW
# Store target value as `Y`
Y = cust_month_tx[target]

In [37]:
Y

CustomerID
12747     1
12748    41
12749     1
12820     0
12822     0
         ..
18280     0
18281     0
18282     0
18283     4
18287     0
Name: 2011-11-01 00:00:00, Length: 3701, dtype: int64

In [ ]:
# Store customer identifier column name as a list
custid = ['CustomerID']

# Select feature column names excluding customer identifier
cols = [col for col in features.columns if col not in custid]

# Extract the features as `X`
X = features[cols]

# Split data to training and testing
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.25, random_state=99)

In [ ]:
# Initialize linear regression instance
linreg = LinearRegression()

# Fit the model to training dataset
linreg.fit(train_X, train_Y)

# Predict the target variable for training data
train_pred_Y = linreg.predict(train_Y)

# Predict the target variable for testing data
test_pred_Y = linreg.predict(test_Y)

In [ ]:
# Calculate root mean squared error on training data
rmse_train = np.sqrt(mean_squared_error(train_Y, train_pred_Y))

# Calculate mean absolute error on training data
mae_train = mean_absolute_error(train_Y, train_pred_Y)

# Calculate root mean squared error on testing data
rmse_test = np.sqrt(mean_squared_error(test_Y, test_pred_Y))

# Calculate mean absolute error on testing data
mae_test = mean_absolute_error(test_Y, test_pred_Y)

# Print the performance metrics
print('RMSE train: {}; RMSE test: {}\nMAE train: {}, MAE test: {}'.format(rmse_train, rmse_test, mae_train, mae_test))

In [ ]:
# Import `statsmodels.api` module
import statsmodels.api as sm

# Initialize model instance on the training data
olsreg = sm.OLS(train_Y, train_X)

# Fit the model
olsreg = olsreg.fit()

# Print model summary
print(olsreg.summary())